In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
import math

In [31]:
# binomial

n_on = 4
n_off = 5
n_tot = n_on+n_off
tau = 5

mu_b = n_off/tau
rho = (1/(1+tau))

p = 1-scp.stats.binom.cdf(n_on-1, n_tot, rho)
print(p)

Z = math.sqrt(2)*scp.special.erfinv(1-2*p)
print(Z)

0.04802149221409335
1.6643476037266856


In [36]:
def LLH(muS, muB, n_off, n_on, tau):
    return scp.stats.poisson.pmf(n_on, (muS+muB)) * scp.stats.poisson.pmf(n_off, (tau*muB))

def negLLHglobal(means, data):
    n_on, n_off, tau = data
    muS, muB = means
    return -LLH(muS, muB, n_off, n_on, tau)

def negLLHprofile(muB, data):
    muS, n_on, n_off, tau = data
    return -LLH(muS, muB, n_off, n_on, tau)

In [38]:
data1 = np.array([n_on, n_off, tau])
globalfit = scp.optimize.minimize(negLLHglobal, args=data1, x0=(1.0,2.0), method='Nelder-Mead')
globalfit

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -0.03428050111526255
             x: [ 3.000e+00  1.000e+00]
           nit: 45
          nfev: 90
 final_simplex: (array([[ 3.000e+00,  1.000e+00],
                       [ 3.000e+00,  1.000e+00],
                       [ 3.000e+00,  1.000e+00]]), array([-3.428e-02, -3.428e-02, -3.428e-02]))

In [39]:
data2 = np.array([0,n_on,n_off,tau])
profilefit = scp.optimize.minimize(negLLHprofile, args=data2, x0=2.0, method='Nelder-Mead')
profilefit

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -0.005147881346465465
             x: [ 1.500e+00]
           nit: 14
          nfev: 28
 final_simplex: (array([[ 1.500e+00],
                       [ 1.500e+00]]), array([-5.148e-03, -5.148e-03]))

In [40]:
LLHratio = -2*np.log(LLH(0,profilefit.x[0],n_off, n_on, tau)/LLH(globalfit.x[0],globalfit.x[1],n_off,n_on,tau))
LLHratio

3.791982941849624

In [41]:
ZPL = math.sqrt(LLHratio)
ZPL

1.9473014512010265

In [64]:
def PBF(j, n_off, rho):
    return ((math.factorial(j+n_off))/(math.factorial(j)*math.factorial(n_off))) * rho**(j)*(1-rho)**(n_off+1)

In [65]:
j = np.linspace(0,n_on-1,n_on, dtype=int)
ps = np.array([PBF(i,n_off,rho) for i in j])
pBF = 1-np.sum(ps)
print(pBF)
ZBF = math.sqrt(2)*scp.special.erfinv(1-2*pBF)
print(ZBF)

0.04802149221409324
1.6643476037266869
